In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
page = requests.get("https://www.indeed.com/jobs?q=data")

In [3]:
soup = BeautifulSoup(page.content, "html.parser")

In [4]:
job_elems = soup.find_all('li', class_='css-5lfssm')
job_elems

[]

In [5]:
for job_elem in job_elems:
    title_elem = job_elem.find('h2', class_='title')
    company_elem = job_elem.find('span', class_='company')
    location_elem = job_elem.find('div', class_='location')
    if None in (title_elem, company_elem, location_elem):
        continue
    print(title_elem.text.strip())
    print(company_elem.text.strip())
    print(location_elem.text.strip())


In [6]:
import requests
from bs4 import BeautifulSoup

# URL de la recherche Indeed
url = "https://www.indeed.com/jobs?q=developer&l=&vjk=3dbc51534ebe66cd"

# Obtenir le contenu HTML de la page
response = requests.get(url)
html_content = response.content

# Analyser le contenu HTML avec Beautiful Soup
soup = BeautifulSoup(html_content, 'lxml')

# Extraire les offres d'emploi
offres_emploi = soup.find_all('div', class_='jobsearch-HeaderContainer')

# Parcourir chaque offre d'emploi
for offre in offres_emploi:
    # Titre de l'offre
    titre = offre.find('a', class_='jobtitle').text.strip()

    # Entreprise
    entreprise = offre.find('span', class_='company').text.strip()

    # Lieu
    lieu = offre.find('span', class_='location').text.strip()

    # Description
    description = offre.find('div', class_='jobdesc').text.strip()

    # URL de l'offre
    lien_offre = offre.find('a', class_='jobtitle')['href']

    # Afficher les informations de l'offre
    print(f"Titre: {titre}")
    print(f"Entreprise: {entreprise}")
    print(f"Lieu: {lieu}")
    print(f"Description: {description}")
    print(f"URL: {lien_offre}\n")


In [7]:
import os
import csv
import requests
from bs4 import BeautifulSoup

headers = {
    "User-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36"}

# Skills and Place of Work
skill = input('Enter your Skill: ').strip()
place = input('Enter the location: ').strip()
no_of_pages = int(input('Enter the #pages to scrape: '))


# Creating the Main Directory
main_dir = os.getcwd() + '\\'
if not os.path.exists(main_dir):
    os.mkdir(main_dir)
    print('Base Directory Created Successfully.')


# Name of the CSV File
file_name = skill.title() + '_' + place.title() + '_Jobs.csv'
# Path of the CSV File
file_path = main_dir + file_name

# Writing to the CSV File
with open(file_path, mode='w') as file:
    writer = csv.writer(file, delimiter=',', lineterminator='\n')
    # Adding the Column Names to the CSV File
    writer.writerow(
        ['JOB_NAME', 'COMPANY', 'LOCATION', 'POSTED', 'APPLY_LINK'])

    # Requesting and getting the webpage using requests
    print(f'\nScraping in progress...\n')
    for page in range(no_of_pages):
        url = 'https://www.indeed.com/jobs?q=' + skill + \
            '&l=' + place + '&start=' + str(page * 10)
        response = requests.get(url, headers=headers)
        html = response.text

        # Scrapping the Web
        soup = BeautifulSoup(html, 'lxml')
        base_url = 'https://in.indeed.com/viewjob?jk='
        d = soup.find('div', attrs={'id': 'mosaic-provider-jobcards'})

        jobs = soup.find_all('a', class_='tapItem')

        for job in jobs:
            job_id = job['id'].split('_')[-1]
            job_title = job.find('span', title=True).text.strip()
            company = job.find('span', class_='companyName').text.strip()
            location = job.find('div', class_='companyLocation').text.strip()
            posted = job.find('span', class_='date').text.strip()
            job_link = base_url + job_id
            #print([job_title, company, location, posted, job_link])

            # Writing to CSV File
            writer.writerow(
                [job_title, company, location.title(), posted, job_link])

print(f'Jobs data written to <{file_name}> successfully.')

Enter your Skill: data
Enter the location: abidjan
Enter the #pages to scrape: 1

Scraping in progress...

Jobs data written to <Data_Abidjan_Jobs.csv> successfully.


In [8]:
import pandas as pd
df = pd.read_csv('Data_Abidjan_Jobs.csv')
df

,JOB_NAME,COMPANY,LOCATION,POSTED,APPLY_LINK


In [9]:
url = 'https://www.indeed.com/jobs?q=data'
# + skill + \
#    '&l=' + place + '&start=' + str(page * 10)
response = requests.get(url, headers=headers)
html = response.text

# Scrapping the Web
soup = BeautifulSoup(html, 'lxml')
base_url = 'https://in.indeed.com/viewjob?jk='
d = soup.find('div', attrs={'id': 'mosaic-provider-jobcards'})

jobs = soup.find_all('a', class_='tapItem')

for job in jobs:
    job_id = job['id'].split('_')[-1]
    job_title = job.find('span', title=True).text.strip()
    company = job.find('span', class_='companyName').text.strip()
    location = job.find('div', class_='companyLocation').text.strip()
    posted = job.find('span', class_='date').text.strip()
    job_link = base_url + job_id
    #print([job_title, company, location, posted, job_link])
    # Afficher les informations de l'offre
    print(f"Titre: {job_title}")
    print(f"Entreprise: {company}")
    print(f"Lieu: {location}")
    print(f"Description: {posted}")
    print(f"URL: {job_link}\n")



In [ ]:
"""
web scraping functionality from www.indeed.com (USA)
"""
import requests
import time
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import re

import pandas as pd

source = "indeed.com"
cookies = {'aep_usuc_f': 'region=US&site=glo&b_locale=en_US&c_tp=USD'}


def get_url(position):
    """
    Generate URL from position and company type: recruiter or direct employer; education level
    """
    url = f"https://indeed.com/jobs?q={position}"

    return url


def get_job_date(card):
    """
     extracts date from the job post record

    :param card:
    :return:
    """
    post_str = card.find('span', {'class': 'date'}).text  # text from the footer: days ago was posted
    post_days = re.findall(r'\d+', post_str)  # extracting number of days from posted_str

    if post_days:
        # calculated date of job posting if days are mentioned
        job_date = (datetime.now() - timedelta(days=int(post_days[0]))).strftime("%d/%m/%Y")
    else:
        job_date = datetime.now().strftime("%d/%m/%Y")  # if days are not mentioned - using today

    return job_date


def get_job_salaries(card):
    """
    extracts salaries
    :param card:
    :return:
    """

    try:
        salary_str = card.find('div', 'metadata salary-snippet-container').text
        salaries = re.findall(r"\b(\w+[.]\w+)", salary_str)

    except AttributeError:
        salaries = []

    return salaries


def get_record(card):
    """
    Extract job data from a single record
    """
    span_tag = card.h2.a.span
    a_tag = card.h2.a

    job_id = a_tag.get("data-jk")  # unique job id
    job_title = span_tag.get("title")  # job title
    job_url = 'https://www.indeed.com' + a_tag.get('href')  # job url
    company_name = card.find('span', {'class': 'companyName'}).text  # company name
    job_loc = card.find('div', {'class': 'companyLocation'}).text  # job location
    job_summary = card.find('div', {'class': 'job-snippet'}).text.strip()  # job description
    job_date = get_job_date(card)  # job posting date
    job_salary = get_job_salaries(card)  # job salaries if any

    record = (job_id, job_title, job_date, job_loc, job_summary, job_salary, job_url, company_name)

    return record


def get_jobs(position):
    """
    creates a DataFrame with all records (scraped jobs), scraping from all pages

    """

    url = get_url(position)
    records = []

    # extract the job data

    while True:

        response = ""
        while response == "":
            try:
                response = requests.get(url=url, cookies=cookies)
                break
            except ConnectionError:
                print("Connection refused by the server..")
                print("Let me sleep for 5 seconds")
                print("ZZzzzz...")
                time.sleep(5)
                print("Was a nice sleep, now let me continue...")
                continue

        soup = BeautifulSoup(response.text, 'html.parser')

        cards = soup.find_all('div', 'job_seen_beacon')

        for card in cards:
            record = get_record(card)
            records.append(record)

        time.sleep(3)  # making a pause before moving to the next page

        # moving to the next page - > assigning a new url
        try:
            url = 'https://indeed.com/' + soup.find('a', {'aria-label': 'Next'}).get('href')

        except AttributeError:
            break

    # save the data as DF
    columns = ['job_id',
               'job_title',
               'job_date',
               'job_loc',
               'job_summary',
               'job_salary',
               'job_url',
               'company_name']
    df = pd.DataFrame(data=records, columns=columns)

    # adding to DF columns with search parameters
    search_time = datetime.now().strftime("%d/%m/%Y, %H:%M:%S")

    df["search_time"] = search_time
    df["search_position"] = position
    df["source"] = source

    return df

In [10]:
"""
web scraping functionality from www.indeed.com (USA)
"""
import requests
import time
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import re

import pandas as pd

source = "indeed.com"
cookies = {'aep_usuc_f': 'region=US&site=glo&b_locale=en_US&c_tp=USD'}

In [11]:
url = f"https://indeed.com/jobs?q=data"